# Executor: LocalCodeExecutor Basic Examples

This notebook shows how to execute Python and shell snippets in **isolated, cached virtual environments**

## Install Required Libraries
Before starting, ensure the required libraries are installed:

In [ ]:
!pip install dapr-agents

## Setup

In [1]:
import logging

from dapr_agents.executors.local import LocalCodeExecutor
from dapr_agents.types.executor import CodeSnippet, ExecutionRequest
from rich.console import Console
from rich.ansi import AnsiDecoder
import shutil

In [2]:
logging.basicConfig(level=logging.INFO)

executor = LocalCodeExecutor()
console = Console()
decoder = AnsiDecoder()

## Running a basic Python Code Snippet

In [3]:
code = """
from rich import print
print("[bold green]Hello executor![/bold green]")
"""

request = ExecutionRequest(snippets=[
    CodeSnippet(language='python', code=code, timeout=10)
])

results = await executor.execute(request)
results[0]  # raw result

# pretty‑print with Rich
console.print(*decoder.decode(results[0].output))

INFO:dapr_agents.executors.local:Sandbox backend enabled: seatbelt
INFO:dapr_agents.executors.local:Created a new virtual environment
INFO:dapr_agents.executors.local:Installing print, rich
INFO:dapr_agents.executors.local:Snippet 1 finished in 2.442s


Hello executor!

## Run a Shell Snipper

In [4]:
shell_request = ExecutionRequest(snippets=[
    CodeSnippet(language='sh', code='echo $((2+2))', timeout=5)
])

await executor.execute(shell_request)

INFO:dapr_agents.executors.local:Sandbox backend enabled: seatbelt
INFO:dapr_agents.executors.local:Snippet 1 finished in 0.019s


[ExecutionResult(status='success', output='4\n', exit_code=0)]

## Reuse the cached virtual environment

In [5]:
# Re‑running the same Python request will reuse the cached venv, so it is faster
await executor.execute(request)

INFO:dapr_agents.executors.local:Sandbox backend enabled: seatbelt
INFO:dapr_agents.executors.local:Reusing cached virtual environment.
INFO:dapr_agents.executors.local:Installing print, rich
INFO:dapr_agents.executors.local:Snippet 1 finished in 0.297s


[ExecutionResult(status='success', output='\x1bHello executor!\x1b\n', exit_code=0)]

## Inject Helper Functions

In [6]:
def fancy_sum(a: int, b: int) -> int:
    return a + b

executor.user_functions.append(fancy_sum)

helper_request = ExecutionRequest(snippets=[
    CodeSnippet(language='python', code='print(fancy_sum(40, 2))', timeout=5)
])

await executor.execute(helper_request)

INFO:dapr_agents.executors.local:Sandbox backend enabled: seatbelt
INFO:dapr_agents.executors.local:Created a new virtual environment
INFO:dapr_agents.executors.local:Snippet 1 finished in 1.408s


[ExecutionResult(status='success', output='42\n', exit_code=0)]

## Clean Up

In [7]:
shutil.rmtree(executor.cache_dir, ignore_errors=True)
print("Cache directory removed ✅")

Cache directory removed ✅


## Package-manager detection & automatic bootstrap

In [8]:
from dapr_agents.executors.utils import package_manager as pm
import pathlib
import tempfile

### Create a throw-away project

In [9]:
tmp_proj = pathlib.Path(tempfile.mkdtemp())
(tmp_proj / "requirements.txt").write_text("rich==13.7.0\n")
print("tmp project:", tmp_proj)

tmp project: /var/folders/9z/8xhqw8x1611fcbhzl339yrs40000gn/T/tmpmssk0m2b


### Show what the helper detects

In [10]:
print("detect_package_managers ->",
      [m.name for m in pm.detect_package_managers(tmp_proj)])
print("get_install_command    ->",
      pm.get_install_command(tmp_proj))

detect_package_managers -> [<PackageManagerType.PIP: 'pip'>]
get_install_command    -> pip install -r requirements.txt


### Point the executor at that directory

In [11]:
import os
from contextlib import contextmanager, ExitStack

@contextmanager
def chdir(path):
    """
    Temporarily change the process CWD to *path*.

    Works on every CPython ≥ 3.6 (and PyPy) and restores the old directory
    even if an exception is raised inside the block.
    """
    old_cwd = os.getcwd()
    os.chdir(path)
    try:
        yield
    finally:
        os.chdir(old_cwd)

In [12]:
with ExitStack() as stack:
    # keep a directory handle open (optional but handy if you’ll delete tmp_proj later)
    stack.enter_context(os.scandir(tmp_proj))

    # <-- our portable replacement for contextlib.chdir()
    stack.enter_context(chdir(tmp_proj))

    # run a trivial snippet; executor will bootstrap because it now “sees”
    # requirements.txt in the current working directory
    out = await executor.execute(
        ExecutionRequest(snippets=[
            CodeSnippet(language="python", code="print('bootstrap OK')", timeout=5)
        ])
    )
    console.print(out[0].output)

INFO:dapr_agents.executors.local:bootstrapping python project with 'pip install -r requirements.txt'
INFO:dapr_agents.executors.local:Sandbox backend enabled: seatbelt
INFO:dapr_agents.executors.local:Created a new virtual environment
INFO:dapr_agents.executors.local:Snippet 1 finished in 1.433s


bootstrap OK

### Clean Up the throw-away project 

In [13]:
shutil.rmtree(executor.cache_dir, ignore_errors=True)
print("Cache directory removed ✅")
shutil.rmtree(tmp_proj, ignore_errors=True)
print("temporary project removed ✅")

Cache directory removed ✅
temporary project removed ✅
